# Using curves with different day-count conventions

(Based on [a question by Vinod Rajakumar](https://sourceforge.net/p/quantlib/mailman/message/36015671/) on the QuantLib mailing list.  Thanks!)

Like a number of other notebooks, this one describes a glitch in the library that you might want to be aware of.

#### The problem

Let's say we're pricing an option.  We've already seen it in other notebooks, so I'll go through the setup without much details:

In [1]:
from QuantLib import *
import math

In [2]:
today = Date(27, July, 2018)
Settings.instance().evaluationDate = today
calendar = UnitedStates(UnitedStates.NYSE)

In [3]:
exercise_date = today+Period(3,Months)
strike = 100.0
option = EuropeanOption(PlainVanillaPayoff(Option.Call, strike),
                        EuropeanExercise(exercise_date))

I'll set up handles for the needed curves, so we can change them later...

In [4]:
u = RelinkableQuoteHandle()
r = RelinkableYieldTermStructureHandle()
sigma = RelinkableBlackVolTermStructureHandle()

In [5]:
process = BlackScholesProcess(u, r, sigma)

...and I'll use the process above to instantiate two different engines: the first uses the analytic Black-Scholes formula, and the second a finite-difference model.

In [6]:
analytic_engine = AnalyticEuropeanEngine(process)

fd_engine = FDEuropeanEngine(process, 1000, 1000)

Now we get to pricing the option.  First, I'll link the risk-free rate and the volatility to two constant curves with the same day-count convention (in this case,  Actual/365 fixed).  Let's say the risk-free rate is 0% and the volatility is 20%.

In [7]:
u.linkTo(SimpleQuote(100.0))
r.linkTo(FlatForward(today, 0.0, Actual365Fixed()))
sigma.linkTo(BlackConstantVol(today, calendar, 0.20, Actual365Fixed()))

With this setup, the two engines give the same result (within numerical error) and everybody is happy:

In [8]:
option.setPricingEngine(analytic_engine)
option.NPV()

4.004101982740124

In [9]:
option.setPricingEngine(fd_engine)
option.NPV()

4.004154055896805

However, things are not always so simple.  For instance, the volatility might have been quoted with a different day-count convention, as is practice on some markets.  Let's say, for instance, that the 20% volatility was quoted based on the commonly used business/252 convention.

In [10]:
sigma.linkTo(BlackConstantVol(today, calendar, 0.20, Business252(calendar)))

In this case, we're not so lucky; the results from the two engines differ significantly.

In [11]:
option.setPricingEngine(analytic_engine)
option.NPV()

4.050510859367279

In [12]:
option.setPricingEngine(fd_engine)
option.NPV()

4.004154055896805

By looking at the numbers, we can see that analytic engine reacts to the change, while the finite-differences engine doesn't.

#### What is happening?

This is not something that could be expected; unfortunately, it's an artifact of the implementation and could only be deduced by looking at the code.  Specifically, the analytic engine is able to include in the calculation the day-count convention of the volatility curve, while the FD model is forced to use one single time grid and can't account for different conventions.

More in detail, what the FD engine does is to ask the curve for the volatility at the exercise date...

In [13]:
vol = sigma.blackVol(exercise_date, strike)
vol

0.2

...and use it on the grid.  However, the time grid on which the FD model works uses the day-count convention of the risk-free rates, resulting in a time to maturity that is inconsistent with the volatility quote...

In [14]:
T_vol = sigma.dayCounter().yearFraction(today, exercise_date)
T_vol

0.25793650793650796

In [15]:
T_grid = r.dayCounter().yearFraction(today, exercise_date)
T_grid

0.25205479452054796

...and therefore the wrong value for the variance of the stock price:

In [16]:
vol*vol*T_grid

0.01008219178082192

In [17]:
var = sigma.blackVariance(exercise_date, strike)
var

0.01031746031746032

#### An attempt at a solution

In this case, and having assessed the problem, we can work around the problem; that is, we can find the volatility that, together with the day-count convention used on the grid, gives the correct variance.

In [18]:
vol = math.sqrt(var/T_grid)
vol

0.20232004929429467

This synthetic value can be used to build a volatility curve with the same day-count convention as the rate.  This allows the FD engine to return a more correct value.

In [19]:
sigma.linkTo(BlackConstantVol(today, calendar, vol, Actual365Fixed()))

In [20]:
option.setPricingEngine(analytic_engine)
option.NPV()

4.050510859367279

In [21]:
option.setPricingEngine(fd_engine)
option.NPV()

4.050563403337715

Of course, this is more cumbersome if the volatility is not flat; you might have to convert multiple values if you're interpolating them, or sample multiple values and then convert them if the curve is of some other kind.

On the whole, it is unfortunate that the implementation is leaking into the use of the engine.  We still don't have a solution, though.  What I can suggest is, when possible, to perform sanity checks like the previous comparison between engine results.  This will give you information on the underlying implementation and the precautions you'll have to take when a comparison is not possible (such as, for instance, when the option is American and there's no corresponding analytic engine).